In [1]:
from fastai2.vision.all import *


In [2]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet

In [3]:
model_config = get_efficientdet_config('tf_efficientdet_d0')

In [4]:
model = EfficientDet(model_config,pretrained_backbone=True)

In [5]:
#model

In [6]:
model_config.num_classes = 1
model_config.image_size = 256

In [7]:
model.class_net = HeadNet(model_config,num_outputs=model_config.num_classes,norm_kwargs=dict(eps=.001,momentum=0.01))

In [8]:
path = Path('/home/heye0507/wheat_detection/data')

In [9]:
# Fastai bbox style is [x1,y1,x2,y2], given coco-style is [x1,y1,w,h]
def convert_fastai_bbox(box):
    x1,y1,w,h = box
    x2 = x1 + w
    y2 = y1 + h
    return [x1,y1,x2,y2]
    
def prep_bbox(df):
    images, bbox = df['image_id'],df['bbox']
    d = collections.defaultdict(list)
    for i,b in zip(images,bbox):
        d[i].append(convert_fastai_bbox([float(i) for i in b[1:-1].split(',')]))
    return d

In [10]:
def get_data(path,sz=224,bs=64):
    base = Path(path).parent
    df = pd.read_csv(path)
    img2bbox = prep_bbox(df)
    dblocks = DataBlock(
        blocks = (ImageBlock,BBoxBlock,BBoxLblBlock),
        splitter = RandomSplitter(valid_pct=0.),
        get_x = lambda o: str(base/'train')+'/'+o+'.jpg',
        get_y = [lambda o: img2bbox[o], lambda o: ['wheat' for i in range(len(img2bbox[o]))]],
        item_tfms = Resize(sz),
        n_inp=1
    )
    return dblocks.dataloaders(img2bbox.keys(),bs=bs)

In [11]:
dls = get_data(path/'arvalis_1.csv',sz=256,bs=4)

In [12]:
b = dls.one_batch()

In [13]:
len(b)

3

In [14]:
b[0].shape,b[1].shape,b[2].shape

(torch.Size([4, 3, 256, 256]), torch.Size([4, 61, 4]), torch.Size([4, 61]))

# unscale
1. fastai bbox is (-1,1)
2. one batch is a tuple with (image, bbox, bboxLabel)
3. unscale on bbox should call on b[1][0:bs]

In [118]:
def un_pad(boxes,labels):
    bb_keep = ((boxes[:,2] - boxes[:,0])>0).nonzero()[:,0]
    return boxes[bb_keep],labels[bb_keep]

In [ ]:
from fastai2.vision.core import _unscale_pnts
from effdet.anchors import Anchors,AnchorLabeler,generate_detections,MAX_DETECTION_POINTS
from effdet.loss import DetectionLoss

In [ ]:
anchors = Anchors(
    model_config.min_level,model_config.max_level,
    model_config.num_scales, model_config.aspect_ratios,
    model_config.anchor_scale, model_config.image_size
    )
anchor_labeler = AnchorLabeler(anchors, model_config.num_classes,match_threshold=0.5)
loss_func = DetectionLoss(model_config)
class_out, box_out = model(b[0])

In [ ]:
boxes, labels = [],[]
perm = torch.LongTensor([1,0,3,2])

for i in range(b[0].shape[0]):
    box, lbl = un_pad(b[1][i],b[2][i])
    boxes.append(_unscale_pnts(box[:,perm],256)) # to tf style, yxyx
    labels.append(lbl.float())

In [ ]:
cls_targets, box_targets, num_positivies = anchor_labeler.batch_label_anchors(
    b[0].shape[0], boxes, labels
)
loss, class_loss, box_loss = loss_func(class_out, box_out, cls_targets, box_targets, num_positivies)
loss,class_loss,box_loss

In [124]:
t_boxs,t_labels = un_pad(b[1][1],b[2][1])

In [125]:
t_boxs.shape,t_labels.shape

(torch.Size([35, 4]), torch.Size([35]))

In [135]:
_unscale_pnts(t_boxs,256)

TensorPoint([[ 42.0000,  98.7500,  62.5000, 117.5000],
        [ 40.2500,  33.7500,  59.7500,  57.0000],
        [ 10.7500,  16.7500,  35.2500,  41.7500],
        [150.7500,  30.5000, 170.2500,  53.0000],
        [167.0000,  29.7500, 192.5000,  72.7500],
        [191.5000,  46.2500, 227.0000,  71.7500],
        [206.2500,  28.0000, 232.7500,  49.5000],
        [233.7500, 184.0000, 256.0000, 200.0000],
        [141.2500,  66.5000, 155.7500,  87.0000],
        [174.5000,  76.2500, 192.5000,  95.0000],
        [192.2500,  87.0000, 215.0000, 106.5000],
        [208.0000, 112.0000, 244.7500, 132.7500],
        [217.2500, 126.2500, 234.5000, 149.2500],
        [146.5000, 144.5000, 176.5000, 163.2500],
        [ 64.2500, 133.5000,  88.0000, 159.0000],
        [ 88.5000, 145.5000, 107.5000, 167.7500],
        [ 25.2500, 153.7500,  42.7500, 178.2500],
        [ 37.0000, 236.5000,  70.0000, 255.2500],
        [116.0000, 194.2500, 139.0000, 222.5000],
        [116.7500, 184.0000, 143.7500, 200.50

In [136]:
_unscale_pnts(t_boxs[:,perm],256)

TensorPoint([[ 98.7500,  42.0000, 117.5000,  62.5000],
        [ 33.7500,  40.2500,  57.0000,  59.7500],
        [ 16.7500,  10.7500,  41.7500,  35.2500],
        [ 30.5000, 150.7500,  53.0000, 170.2500],
        [ 29.7500, 167.0000,  72.7500, 192.5000],
        [ 46.2500, 191.5000,  71.7500, 227.0000],
        [ 28.0000, 206.2500,  49.5000, 232.7500],
        [184.0000, 233.7500, 200.0000, 256.0000],
        [ 66.5000, 141.2500,  87.0000, 155.7500],
        [ 76.2500, 174.5000,  95.0000, 192.5000],
        [ 87.0000, 192.2500, 106.5000, 215.0000],
        [112.0000, 208.0000, 132.7500, 244.7500],
        [126.2500, 217.2500, 149.2500, 234.5000],
        [144.5000, 146.5000, 163.2500, 176.5000],
        [133.5000,  64.2500, 159.0000,  88.0000],
        [145.5000,  88.5000, 167.7500, 107.5000],
        [153.7500,  25.2500, 178.2500,  42.7500],
        [236.5000,  37.0000, 255.2500,  70.0000],
        [194.2500, 116.0000, 222.5000, 139.0000],
        [184.0000, 116.7500, 200.5000, 143.75

In [127]:
b[1][1]

tensor([[-0.6719, -0.2285, -0.5117, -0.0820],
        [-0.6855, -0.7363, -0.5332, -0.5547],
        [-0.9160, -0.8691, -0.7246, -0.6738],
        [ 0.1777, -0.7617,  0.3301, -0.5859],
        [ 0.3047, -0.7676,  0.5039, -0.4316],
        [ 0.4961, -0.6387,  0.7734, -0.4395],
        [ 0.6113, -0.7812,  0.8184, -0.6133],
        [ 0.8262,  0.4375,  1.0000,  0.5625],
        [ 0.1035, -0.4805,  0.2168, -0.3203],
        [ 0.3633, -0.4043,  0.5039, -0.2578],
        [ 0.5020, -0.3203,  0.6797, -0.1680],
        [ 0.6250, -0.1250,  0.9121,  0.0371],
        [ 0.6973, -0.0137,  0.8320,  0.1660],
        [ 0.1445,  0.1289,  0.3789,  0.2754],
        [-0.4980,  0.0430, -0.3125,  0.2422],
        [-0.3086,  0.1367, -0.1602,  0.3105],
        [-0.8027,  0.2012, -0.6660,  0.3926],
        [-0.7109,  0.8477, -0.4531,  0.9941],
        [-0.0938,  0.5176,  0.0859,  0.7383],
        [-0.0879,  0.4375,  0.1230,  0.5664],
        [ 0.2129,  0.6758,  0.4102,  0.8027],
        [ 0.5254,  0.6777,  0.7168

In [110]:
b[1].shape

torch.Size([4, 61, 4])

In [104]:
t_boxs[1].shape

torch.Size([61, 4])

In [107]:
b[1][1]

tensor([[-0.6719, -0.2285, -0.5117, -0.0820],
        [-0.6855, -0.7363, -0.5332, -0.5547],
        [-0.9160, -0.8691, -0.7246, -0.6738],
        [ 0.1777, -0.7617,  0.3301, -0.5859],
        [ 0.3047, -0.7676,  0.5039, -0.4316],
        [ 0.4961, -0.6387,  0.7734, -0.4395],
        [ 0.6113, -0.7812,  0.8184, -0.6133],
        [ 0.8262,  0.4375,  1.0000,  0.5625],
        [ 0.1035, -0.4805,  0.2168, -0.3203],
        [ 0.3633, -0.4043,  0.5039, -0.2578],
        [ 0.5020, -0.3203,  0.6797, -0.1680],
        [ 0.6250, -0.1250,  0.9121,  0.0371],
        [ 0.6973, -0.0137,  0.8320,  0.1660],
        [ 0.1445,  0.1289,  0.3789,  0.2754],
        [-0.4980,  0.0430, -0.3125,  0.2422],
        [-0.3086,  0.1367, -0.1602,  0.3105],
        [-0.8027,  0.2012, -0.6660,  0.3926],
        [-0.7109,  0.8477, -0.4531,  0.9941],
        [-0.0938,  0.5176,  0.0859,  0.7383],
        [-0.0879,  0.4375,  0.1230,  0.5664],
        [ 0.2129,  0.6758,  0.4102,  0.8027],
        [ 0.5254,  0.6777,  0.7168

In [31]:
#dls.decode_batch(b)[0][1]

In [18]:
from fastai2.vision.core import _unscale_pnts

In [96]:
_unscale_pnts(b[1][1],256)

TensorPoint([[ 42.0000,  98.7500,  62.5000, 117.5000],
        [ 40.2500,  33.7500,  59.7500,  57.0000],
        [ 10.7500,  16.7500,  35.2500,  41.7500],
        [150.7500,  30.5000, 170.2500,  53.0000],
        [167.0000,  29.7500, 192.5000,  72.7500],
        [191.5000,  46.2500, 227.0000,  71.7500],
        [206.2500,  28.0000, 232.7500,  49.5000],
        [233.7500, 184.0000, 256.0000, 200.0000],
        [141.2500,  66.5000, 155.7500,  87.0000],
        [174.5000,  76.2500, 192.5000,  95.0000],
        [192.2500,  87.0000, 215.0000, 106.5000],
        [208.0000, 112.0000, 244.7500, 132.7500],
        [217.2500, 126.2500, 234.5000, 149.2500],
        [146.5000, 144.5000, 176.5000, 163.2500],
        [ 64.2500, 133.5000,  88.0000, 159.0000],
        [ 88.5000, 145.5000, 107.5000, 167.7500],
        [ 25.2500, 153.7500,  42.7500, 178.2500],
        [ 37.0000, 236.5000,  70.0000, 255.2500],
        [116.0000, 194.2500, 139.0000, 222.5000],
        [116.7500, 184.0000, 143.7500, 200.50

In [86]:
perm = torch.LongTensor([1,0,3,2])

In [91]:
_unscale_pnts(b[1],256)[:,:,perm][0][0]

tensor([ 95.0000,   0.0000, 108.7500,   3.2500])

In [33]:
from effdet.anchors import Anchors,AnchorLabeler,generate_detections,MAX_DETECTION_POINTS
from effdet.loss import DetectionLoss

In [36]:
anchors = Anchors(
    model_config.min_level,model_config.max_level,
    model_config.num_scales, model_config.aspect_ratios,
    model_config.anchor_scale, model_config.image_size
    )

In [38]:
anchor_labeler = AnchorLabeler(anchors, model_config.num_classes,match_threshold=0.5)

In [39]:
loss_func = DetectionLoss(model_config)

In [40]:
preds = model(b[0])
class_out, box_out = preds

In [129]:
b[0].shape[0]

4

In [140]:
boxes, labels = [],[]
perm = torch.LongTensor([1,0,3,2])

for i in range(b[0].shape[0]):
    box, lbl = un_pad(b[1][i],b[2][i])
    boxes.append(_unscale_pnts(box[:,perm],256)) # to tf style, yxyx
    labels.append(lbl.float())

In [143]:
cls_targets, box_targets, num_positivies = anchor_labeler.batch_label_anchors(
    b[0].shape[0], boxes, labels
)

In [144]:
loss, class_loss, box_loss = loss_func(class_out, box_out, cls_targets, box_targets, num_positivies)

In [145]:
loss,class_loss,box_loss

(tensor(19.9048, grad_fn=<AddBackward0>),
 tensor(17.6527, grad_fn=<SumBackward1>),
 tensor(0.0450, grad_fn=<SumBackward1>))

In [54]:
loss

tensor(20.1627, grad_fn=<AddBackward0>)

In [56]:
class_loss

tensor(17.5499, grad_fn=<SumBackward1>)

In [57]:
box_loss

tensor(0.0523, grad_fn=<SumBackward1>)

In [65]:
box_targets[4].shape

torch.Size([4, 2, 2, 36])

In [21]:
b1 = dls.one_batch()
b1[0].shape,b1[1].shape,b1[2].shape

(torch.Size([4, 3, 256, 256]), torch.Size([4, 66, 4]), torch.Size([4, 66]))

In [28]:
preds = model(b[0])

In [29]:
for p in preds[0]:
    print(p.shape)

torch.Size([4, 9, 32, 32])
torch.Size([4, 9, 16, 16])
torch.Size([4, 9, 8, 8])
torch.Size([4, 9, 4, 4])
torch.Size([4, 9, 2, 2])


In [30]:
for p in preds[1]:
    print(p.shape)

torch.Size([4, 36, 32, 32])
torch.Size([4, 36, 16, 16])
torch.Size([4, 36, 8, 8])
torch.Size([4, 36, 4, 4])
torch.Size([4, 36, 2, 2])
